
### 🔄 Repository Update Script (Colab-Compatible)

This script automates the process of keeping the `ich-modeling-2025` repository up to date in **Google Colab**, with built-in protection for local modifications.

#### ✅ What the script does:

1. **Mounts Google Drive** so the repository is accessible in your workspace.
2. **Checks if the repository already exists** in `/MyDrive/ich-modeling-2025`.
3. If the repository **exists**:
   - It checks for any **locally modified or untracked files**.
   - These files are **backed up** into a new folder named `Backup_YYYYMMDD_HHMM` in `/MyDrive/`.
   - Then, it runs a **forced pull** by:
     - Fetching the latest changes from the remote repository.
     - Resetting the local copy to match the remote `main` branch exactly (`git reset --hard origin/main`).
4. If the repository **does not exist**, it clones the repo from GitHub.

#### 📁 Backup Behavior:

- Only files marked as **modified** or **untracked** by Git are backed up.
- Backups preserve folder structure and are stored outside the repo directory.
- This ensures **no local work is lost**, even when a hard reset is performed.

#### 🔧 Assumptions:

- The GitHub repo is: `https://github.com/PriyeshGosai/ich-modeling-2025`
- The default branch is `main`. Modify this in the script if your default branch is `master` or something else.

In [ ]:
from google.colab import drive
import os
import subprocess
import shutil
from datetime import datetime

# Mount Google Drive
drive.mount('/content/drive')

# Base directory
base_dir = '/content/drive/MyDrive/'
os.chdir(base_dir)

FOLDER = 'ich-modeling-2025'
REPO_URL = f'https://github.com/PriyeshGosai/{FOLDER}'
repo_path = os.path.join(base_dir, FOLDER)

# Get timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
backup_folder = f'Backup_{timestamp}'
backup_path = os.path.join(base_dir, backup_folder)

# If repo exists, back up and force pull
if os.path.exists(repo_path):
    os.chdir(repo_path)

    # Get list of modified and untracked files
    result = subprocess.run(['git', 'status', '--porcelain'], stdout=subprocess.PIPE, text=True)
    changed_files = [line[3:] for line in result.stdout.splitlines() if line and line[0] in ('M', 'A', '??')]

    if changed_files:
        os.makedirs(backup_path, exist_ok=True)
        print(f"Backing up {len(changed_files)} changed files to {backup_folder}...")
        for file in changed_files:
            src = os.path.join(repo_path, file)
            dst = os.path.join(backup_path, file)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            if os.path.exists(src):
                shutil.copy2(src, dst)

    # Force pull: reset local repo to match remote
    subprocess.run(['git', 'fetch', 'origin'])
    subprocess.run(['git', 'reset', '--hard', 'origin/main'])

else:
    os.chdir(base_dir)
    subprocess.run(['git', 'clone', REPO_URL])

